In [1]:
import os
import sys
from pathlib import Path

import torch
from torch.ao.quantization.quantize_pt2e import convert_pt2e, prepare_pt2e
from torch.ao.quantization.quantizer.x86_inductor_quantizer import (
    X86InductorQuantizer,
    get_default_x86_inductor_quantization_config,
)
from torch.utils.data import DataLoader
from torch._export import capture_pre_autograd_graph

sys.path.append(os.fspath(Path.cwd().parent.resolve()))

In [2]:
from anime4k_torch.data import NpyDataset, transform_image_eval
from anime4k_torch.model import SR2Model

In [3]:
data_dir = Path("../../data/synla").resolve()
validation_set = NpyDataset(data_dir / "synla_1024.npy", transform_image_eval)
validation_loader = DataLoader(validation_set, batch_size=1)

In [4]:
checkpoint_dir = Path("../checkpoints").resolve()

In [5]:
model = SR2Model(highway_depth=4, block_depth=7)
model.load_state_dict(torch.load(checkpoint_dir / "synla_f32.pth"))
model.eval()
model = capture_pre_autograd_graph(model, (next(iter(validation_loader))[0],))
quantizer = X86InductorQuantizer().set_global(
    get_default_x86_inductor_quantization_config()
)
model = prepare_pt2e(model, quantizer)
with torch.inference_mode():
    for images, _ in validation_loader:
        model(images)
model = convert_pt2e(model)

In [6]:
model.print_readable()
None

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: "f32[1, 3, 128, 128]"; 
    
        arg0, = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        arg0_1 = arg0
        
        # No stacktrace found for following nodes
        quantize_per_tensor_default = torch.ops.quantized_decomposed.quantize_per_tensor.default(arg0_1, 0.003921329043805599, 0, 0, 255, torch.uint8)
        
        # File: /home/yunhao/ECE532-Digital-Systems-Design/ECE532/anime4k_torch/anime4k_torch/model.py:83, code: x = conv_layer(x)
        dequantize_per_tensor_default = torch.ops.quantized_decomposed.dequantize_per_tensor.default(quantize_per_tensor_default, 0.003921329043805599, 0, 0, 255, torch.uint8);  quantize_per_tensor_default = None
        
        # File: /home/yunhao/ECE532-Digital-Systems-Design/ECE532/anime4k_torch/anime4k_torch/model.py:76, code: upsampled_x = F.interpolate(x, scale_factor=2, mode="bilinear")
        upsample_bilinear2d: "f32[1, 3, 256, 256]" = t